<a href="https://colab.research.google.com/github/AJamal27891/University-coursees-projects/blob/master/distilbert_Base_Matching_entities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# download dependencies

In [2]:
!rm -r  test_trainer 

rm: cannot remove 'test_trainer': No such file or directory


In [3]:
!pip install transformers
!pip install datasets

## load datasets 

In [6]:
dirty_walmart_amazon = r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/dirty_walmart_amazon'
dirty_dblp_scholar = r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/dirty_dblp_scholar'
dirty_amazon_ituns = r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/dirty_amazon_itunes'
dirty_dblp_acm = r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/dirty_dblp_acm'
abt_buy = r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/abt_buy'
amazon_google = r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/amazon_google'
QQP= r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/QQP'

## Test dataset merger functions

In [4]:
import pandas as pd 
data_A = pd.DataFrame({'id':[1,2,3,4,5,6],'price':[.2,.5,.6,.7,1,.8],'describtion':['samsung s5','iphone','xperia','sonny','hstc','xomi']})
data_B = pd.DataFrame({'id':[1,2,3,4,5,6],'price':[.2,.5,.6,.7,1,.8],'describtion':['iphone','xperia','sonny','huawi','xomi','samsung s5']})
lables = pd.DataFrame({'left_id':[1,2,3,4,5,6],'right_id':[6,1,2,3,4,5],'label':[1,1,1,1,0,1]})

In [5]:
total_data = lables.merge(data_A,left_on='left_id',right_on='id')
total_data = total_data.merge(data_B,left_on='right_id',right_on='id')
total_data

,left_id,right_id,label,id_x,price_x,describtion_x,id_y,price_y,describtion_y
0,1,6,1,1,0.2,samsung s5,6,0.8,samsung s5
1,2,1,1,2,0.5,iphone,1,0.2,iphone
2,3,2,1,3,0.6,xperia,2,0.5,xperia
3,4,3,1,4,0.7,sonny,3,0.6,sonny
4,5,4,0,5,1.0,hstc,4,0.7,huawi
5,6,5,1,6,0.8,xomi,5,1.0,xomi


In [ ]:
import random 
total_data


,left_id,right_id,label,id_x,price_x,describtion_x,id_y,price_y,describtion_y
0,1,6,1,1,0.2,samsung s5,6,0.8,samsung s5
1,2,1,1,2,0.5,iphone,1,0.2,iphone
2,3,2,1,3,0.6,xperia,2,0.5,xperia
3,4,3,1,4,0.7,sonny,3,0.6,sonny
4,5,4,0,5,1.0,hstc,4,0.7,huawi
5,6,5,1,6,0.8,xomi,5,1.0,xomi


In [7]:
def remove_id_columns(total_data):
  for x in total_data:
    if x.endswith('id'):
      total_data.drop(x,axis=1,inplace=True)
    if x.startswith('id'):
      total_data.drop(x,axis=1,inplace=True)
  return total_data  
remove_id_columns(total_data)

,label,price_x,describtion_x,price_y,describtion_y
0,1,0.2,samsung s5,0.8,samsung s5
1,1,0.5,iphone,0.2,iphone
2,1,0.6,xperia,0.5,xperia
3,1,0.7,sonny,0.6,sonny
4,0,1.0,hstc,0.7,huawi
5,1,0.8,xomi,1.0,xomi


In [ ]:
total_data =remove_id_columns(total_data)

In [ ]:
total_data.astype(str).apply(lambda x : ' <atr>'+x.name.strip('_y').strip('_x') +'<val>'+x)

,label,price_x,describtion_x,price_y,describtion_y
0,<atr>label<val>1,<atr>price<val>0.2,<atr>describtion<val>samsung s5,<atr>price<val>0.8,<atr>describtion<val>samsung s5
1,<atr>label<val>1,<atr>price<val>0.5,<atr>describtion<val>iphone,<atr>price<val>0.2,<atr>describtion<val>iphone
2,<atr>label<val>1,<atr>price<val>0.6,<atr>describtion<val>xperia,<atr>price<val>0.5,<atr>describtion<val>xperia
3,<atr>label<val>1,<atr>price<val>0.7,<atr>describtion<val>sonny,<atr>price<val>0.6,<atr>describtion<val>sonny
4,<atr>label<val>0,<atr>price<val>1.0,<atr>describtion<val>hstc,<atr>price<val>0.7,<atr>describtion<val>huawi
5,<atr>label<val>1,<atr>price<val>0.8,<atr>describtion<val>xomi,<atr>price<val>1.0,<atr>describtion<val>xomi


In [ ]:
isting = [1,2,3,4,5]
elment = [5]
elment + isting

[5, 1, 2, 3, 4, 5]

In [8]:
import random
def augmentation(df):
  first = ['label']
  clist = df.columns.tolist()
  clist.remove(first[0])
  random.shuffle(clist)
  df1 = df[first+clist]
  random.shuffle(clist)
  df2 = df[first+clist]
  random.shuffle(clist)
  df3 = df[first+clist]
  random.shuffle(clist)
  df4 = df[first+clist]
  return df, df1,df2,df3,df4


In [ ]:
df1,df2,df3,df4,df5 = augmentation(total_data)
df1

,label,price_x,describtion_x,price_y,describtion_y
0,1,0.2,samsung s5,0.8,samsung s5
1,1,0.5,iphone,0.2,iphone
2,1,0.6,xperia,0.5,xperia
3,1,0.7,sonny,0.6,sonny
4,0,1.0,hstc,0.7,huawi
5,1,0.8,xomi,1.0,xomi


In [9]:
def remove_id_columns(total_data):
  for x in total_data:
    if x.endswith('id'):
      total_data.drop(x,axis=1,inplace=True)
    if x.startswith('id'):
      total_data.drop(x,axis=1,inplace=True)
  return total_data  
remove_id_columns(total_data)

def merger(data_A,data_B,train,test,valid,
           remove_ids=True,
           merge_attributes=True,
           drop_original=True,
           tuples=True):
  train_data = train.merge(data_A,left_on='ltable_id',right_on='id')
  train_data = train_data.merge(data_B,left_on='rtable_id',right_on='id')
  test_data = test.merge(data_A,left_on='ltable_id',right_on='id')
  test_data = test_data.merge(data_B,left_on='rtable_id',right_on='id')
  valid_data = valid.merge(data_A,left_on='ltable_id',right_on='id')
  valid_data = valid_data.merge(data_B,left_on='rtable_id',right_on='id')
  if remove_ids:
    train_data = remove_id_columns(train_data)
    test_data = remove_id_columns(test_data)
    valid_data = remove_id_columns(valid_data)
  if merge_attributes:
    dfs = augmentation(train_data)
    new_dfs = []
    for df in dfs:
      df.iloc[:,1:]  = train_data.iloc[:,1:].astype(str).apply(lambda x : ' [COL] '+x.name.strip('_y').strip('_x') +' [VAL] '+x)
      df['text_left'] =train_data[[c for c in train_data.columns[1:] if c.endswith('_x')]].apply(lambda x :" [CLS] " + ' '.join(x.astype(str)) ,axis=1)
      df['text_right'] =train_data[[c for c in train_data.columns[1:] if c.endswith('_y')]].apply(lambda x :" [SEP] " + ' '.join(x.astype(str)) ,axis=1)
      new_dfs.append(df[['label','text_left','text_right']])
    train_data = pd.concat(new_dfs)
    
      
    test_data.iloc[:,1:]  = test_data.iloc[:,1:].astype(str).apply(lambda x : ' [COL] '+x.name.strip('_y').strip('_x') +' [VAL] '+x)
    test_data['text_left'] =test_data[[c for c in test_data.columns[1:] if c.endswith('_x')]].apply(lambda x :" [CLS] " + ' '.join(x.astype(str)) ,axis=1)
    test_data['text_right'] =test_data[[c for c in test_data.columns[1:] if c.endswith('_y')]].apply(lambda x :" [SEP] " + ' '.join(x.astype(str)) ,axis=1)
    valid_data.iloc[:,1:]  = valid_data.iloc[:,1:].astype(str).apply(lambda x : ' [COL]'+x.name.strip('_y').strip('_x') +' [VAL] '+x)
    valid_data['text_left'] =valid_data[[c for c in valid_data.columns[1:] if c.endswith('_x')]].apply(lambda x :" [CLS] " + ' '.join(x.astype(str)) ,axis=1)
    valid_data['text_right'] =valid_data[[c for c in valid_data.columns[1:] if c.endswith('_y')]].apply(lambda x :" [SEP] " + ' '.join(x.astype(str)) ,axis=1) 

  if drop_original:
    train_data= train_data[['label','text_left','text_right']]
    test_data= test_data[['label','text_left','text_right']]
    valid_data= valid_data[['label','text_left','text_right']]
  if tuples:
    train_data['tuples']= train_data[['text_left','text_right']].values.tolist()
    train_data['tuples'] = train_data['tuples'].apply(lambda x: ' '.join(x))
    test_data['tuples']= test_data[['text_left','text_right']].values.tolist()
    test_data['tuples'] = test_data['tuples'].apply(lambda x: ' '.join(x))
    valid_data['tuples']= valid_data[['text_left','text_right']].values.tolist()
    valid_data['tuples'] = valid_data['tuples'].apply(lambda x: ' '.join(x))

  return train_data, test_data, valid_data

# Data Preprocessing

## Amazon Google data

In [ ]:
table_A = pd.read_csv(amazon_google+r'/deep_matcher/tableA.csv')
table_B = pd.read_csv(amazon_google+r'/deep_matcher/tableB.csv')
AG_train = pd.read_csv(amazon_google+r'/deep_matcher/train.csv')
AG_test = pd.read_csv(amazon_google+r'/deep_matcher/test.csv')
AG_valid = pd.read_csv(amazon_google+r'/deep_matcher/valid.csv')
AG_train, AG_test, AG_valid = merger(table_A,table_B,AG_train,AG_test,AG_valid)
AG_train.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1715: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,label,text_left,text_right,tuples
0,0,[CLS] [COL] title [VAL] microsoft visio stan...,[SEP] [COL] title [VAL] adobe cs3 design sta...,[CLS] [COL] title [VAL] microsoft visio stan...
1,0,[CLS] [COL] title [VAL] adobe premiere pro c...,[SEP] [COL] title [VAL] adobe cs3 design sta...,[CLS] [COL] title [VAL] adobe premiere pro c...
2,0,[CLS] [COL] title [VAL] adobe indesign cs3 u...,[SEP] [COL] title [VAL] adobe cs3 design sta...,[CLS] [COL] title [VAL] adobe indesign cs3 u...
3,0,[CLS] [COL] title [VAL] adobe creative suite...,[SEP] [COL] title [VAL] adobe cs3 design sta...,[CLS] [COL] title [VAL] adobe creative suite...
4,0,[CLS] [COL] title [VAL] adobe creative suite...,[SEP] [COL] title [VAL] adobe cs3 design sta...,[CLS] [COL] title [VAL] adobe creative suite...


In [ ]:
AG_train['tuples'][0]

0     [CLS]  [COL] title [VAL] microsoft visio stan...
0     [CLS]  [COL] title [VAL] microsoft visio stan...
0     [CLS]  [COL] title [VAL] microsoft visio stan...
0     [CLS]  [COL] title [VAL] microsoft visio stan...
0     [CLS]  [COL] title [VAL] microsoft visio stan...
Name: tuples, dtype: object

In [ ]:
AG_train['label'].describe()

count    34370.000000
mean         0.101688
std          0.302241
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: label, dtype: float64

In [ ]:
AG_train = AG_train.groupby('label')
AG_train = AG_train.apply(lambda x: x.sample(AG_train.size().min()).reset_index(drop=True))
AG_train = AG_train.sample(frac=1).reset_index(drop=True)
AG_test = AG_test.groupby('label')
AG_test = AG_test.apply(lambda x: x.sample(AG_test.size().min()).reset_index(drop=True))
AG_test = AG_test.sample(frac=1).reset_index(drop=True)
AG_valid = AG_valid.groupby('label')
AG_valid = AG_valid.apply(lambda x: x.sample(AG_valid.size().min()).reset_index(drop=True))
AG_valid = AG_valid.sample(frac=1).reset_index(drop=True)

In [ ]:
import os 
try:
  os.mkdir('merged_data')
  os.mkdir('merged_data/amazon_google')
except FileExistsError:
  pass

AG_train.to_csv(r'merged_data/amazon_google/train.csv')
AG_test.to_csv(r'merged_data/amazon_google/test.csv')
AG_valid.to_csv(r'merged_data/amazon_google/valid.csv')



In [ ]:
from datasets import load_dataset
raw_datasets = load_dataset('csv',data_files={'train':['merged_data/amazon_google/train.csv','merged_data/amazon_google/valid.csv'],
                                              'test':'merged_data/amazon_google/test.csv',
                                              })

Using custom data configuration default-975b63265696902c


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-975b63265696902c/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


In [16]:
from transformers import DistilBertTokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-cased",
                                                # additional_special_tokens = ['[COL]','[VAL]']
                                                )

In [17]:
maxlen = 128

# tokenizer.eos_token ='[SEP]'

# tokenizer.bos_token = '[CLS]'

# tokenizer.additional_special_tokens = ['[COL]','[VAL]']

def tokenize_function(examples):
    return tokenizer(examples['tuples'], padding="max_length", truncation=True,max_length=maxlen)
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns('Unnamed: 0')
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'text_left', 'text_right', 'tuples'],
        num_rows: 7458
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'text_left', 'text_right', 'tuples'],
        num_rows: 468
    })
})

In [27]:
import torch
torch.cuda.empty_cache()

In [28]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier

In [29]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer",num_train_epochs=12,per_device_train_batch_size=16)

In [ ]:
tokenizer.decode(tokenized_datasets['train']['input_ids'][1])

'[CLS] [CLS] [COL] title [VAL] quickbooks pro 2007 for mac ( mac ) [COL] manufacturer [VAL] intuit [COL] price [VAL] 199. 95 [SEP] [COL] title [VAL] quickbooks pro 2007 3 user [COL] manufacturer [VAL] nan [COL] price [VAL] 409. 52 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [37]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=tokenized_datasets['train'], eval_dataset=tokenized_datasets['test']
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: tuples, text_right, text_left.
***** Running training *****
  Num examples = 7458
  Num Epochs = 12
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 5604


Step,Training Loss
500,0.393200
1000,0.206700
1500,0.137900
2000,0.095900
2500,0.071600
3000,0.063300
3500,0.041200
4000,0.027600
4500,0.026300
5000,0.015300


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1500
Configuration saved in test_trainer/checkpoint-1500/config.json
Model weights saved in test_trainer/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2000
Configuration saved in test_trainer/checkpoint-2000/config.json
Model weights saved in test_trainer/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2500
Configuration saved in test_trainer/checkpoint-2500/config.json
Model weights saved in test_trainer/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-30

TrainOutput(global_step=5604, training_loss=0.09730879968825959, metrics={'train_runtime': 692.9998, 'train_samples_per_second': 129.143, 'train_steps_per_second': 8.087, 'total_flos': 4602013881569280.0, 'train_loss': 0.09730879968825959, 'epoch': 12.0})

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
tokenizer.decode(tokenized_datasets['train']['input_ids'][0])

'[CLS] [CLS] [COL] title [VAL] pc tools spyware doctor 2007 [COL] manufacturer [VAL] encore [COL] price [VAL] 29. 95 [SEP] [COL] title [VAL] encore software 10990 pc tools spyware doctor 2007 win 98 me 2000 xp [COL] manufacturer [VAL] nan [COL] price [VAL] 26. 97 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: tuples, text_right, text_left.
***** Running Evaluation *****
  Num examples = 468
  Batch size = 8


{'eval_f1': 0.8617511520737327,
 'eval_loss': 1.1027989387512207,
 'eval_runtime': 1.3132,
 'eval_samples_per_second': 356.381,
 'eval_steps_per_second': 44.928}

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: tuples, text_right, text_left.
***** Running Evaluation *****
  Num examples = 468
  Batch size = 8


{'eval_accuracy': 0.8717948717948718,
 'eval_loss': 1.1027989387512207,
 'eval_runtime': 1.2664,
 'eval_samples_per_second': 369.548,
 'eval_steps_per_second': 46.588}

## abt-Buy dataset

In [46]:
table_A = pd.read_csv(abt_buy+r'/deep_matcher/tableA.csv')
table_B = pd.read_csv(abt_buy+r'/deep_matcher/tableB.csv')
AG_train = pd.read_csv(abt_buy+r'/deep_matcher/train.csv')
AG_test = pd.read_csv(abt_buy+r'/deep_matcher/test.csv')
AG_valid = pd.read_csv(abt_buy+r'/deep_matcher/valid.csv')
AG_train, AG_test, AG_valid = merger(table_A,table_B,AG_train,AG_test,AG_valid)
AG_train.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1715: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,label,text_left,text_right,tuples
0,0,[CLS] [COL] name [VAL] lg 24 ' lds4821ww sem...,[SEP] [COL] name [VAL] lg ldf6920bb fully in...,[CLS] [COL] name [VAL] lg 24 ' lds4821ww sem...
1,0,[CLS] [COL] name [VAL] lg 24 ' lds4821bb sem...,[SEP] [COL] name [VAL] lg ldf6920bb fully in...,[CLS] [COL] name [VAL] lg 24 ' lds4821bb sem...
2,1,[CLS] [COL] name [VAL] lg 24 ' ldf6920bb ful...,[SEP] [COL] name [VAL] lg ldf6920bb fully in...,[CLS] [COL] name [VAL] lg 24 ' ldf6920bb ful...
3,0,[CLS] [COL] name [VAL] lg 24 ' ldf6920ww ful...,[SEP] [COL] name [VAL] lg ldf6920bb fully in...,[CLS] [COL] name [VAL] lg 24 ' ldf6920ww ful...
4,0,[CLS] [COL] name [VAL] lg 24 ' ldf6920st ful...,[SEP] [COL] name [VAL] lg ldf6920bb fully in...,[CLS] [COL] name [VAL] lg 24 ' ldf6920st ful...


In [47]:
AG_train = AG_train.groupby('label')
AG_train = AG_train.apply(lambda x: x.sample(AG_train.size().min()).reset_index(drop=True))
AG_train = AG_train.sample(frac=1).reset_index(drop=True)
AG_test = AG_test.groupby('label')
AG_test = AG_test.apply(lambda x: x.sample(AG_test.size().min()).reset_index(drop=True))
AG_test = AG_test.sample(frac=1).reset_index(drop=True)
AG_valid = AG_valid.groupby('label')
AG_valid = AG_valid.apply(lambda x: x.sample(AG_valid.size().min()).reset_index(drop=True))
AG_valid = AG_valid.sample(frac=1).reset_index(drop=True)

In [48]:
import os 
try:
  os.mkdir('merged_data/abt_buy/')
except FileExistsError:
  pass

AG_train.to_csv(r'merged_data/abt_buy/train.csv')
AG_test.to_csv(r'merged_data/abt_buy/test.csv')
AG_valid.to_csv(r'merged_data/abt_buy/valid.csv')


In [49]:
from datasets import load_dataset
raw_datasets = load_dataset('csv',data_files={'train':['merged_data/abt_buy/train.csv','merged_data/abt_buy/valid.csv'],
                                              'test':'merged_data/abt_buy/test.csv',
                                              })

Using custom data configuration default-75a4c31827c7c2d2


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-75a4c31827c7c2d2/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


In [34]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns('Unnamed: 0')
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'text_left', 'text_right', 'tuples'],
        num_rows: 6572
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'text_left', 'text_right', 'tuples'],
        num_rows: 412
    })
})

In [50]:
import torch 
from transformers import AutoModelForSequenceClassification
torch.cuda.empty_cache()
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
training_args = TrainingArguments("here",num_train_epochs=14,per_device_train_batch_size=16)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.8.2",
  "vocab_size": 30522
}

loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9a

In [51]:
tokenizer.decode(tokenized_datasets['train']['input_ids'][1])
trainer = Trainer(
    model=model, args=training_args, train_dataset=tokenized_datasets['train'], eval_dataset=tokenized_datasets['test']
)
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text_left, tuples, text_right.
***** Running training *****
  Num examples = 6146
  Num Epochs = 14
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 5390


Step,Training Loss
500,0.452800


Saving model checkpoint to here/checkpoint-500
Configuration saved in here/checkpoint-500/config.json
Model weights saved in here/checkpoint-500/pytorch_model.bin


Step,Training Loss
500,0.452800
1000,0.189700
1500,0.123800
2000,0.073200
2500,0.058600
3000,0.038200
3500,0.023400
4000,0.014000
4500,0.011100
5000,0.008200


Saving model checkpoint to here/checkpoint-1000
Configuration saved in here/checkpoint-1000/config.json
Model weights saved in here/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to here/checkpoint-1500
Configuration saved in here/checkpoint-1500/config.json
Model weights saved in here/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to here/checkpoint-2000
Configuration saved in here/checkpoint-2000/config.json
Model weights saved in here/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to here/checkpoint-2500
Configuration saved in here/checkpoint-2500/config.json
Model weights saved in here/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to here/checkpoint-3000
Configuration saved in here/checkpoint-3000/config.json
Model weights saved in here/checkpoint-3000/pytorch_model.bin
Saving model checkpoint to here/checkpoint-3500
Configuration saved in here/checkpoint-3500/config.json
Model weights saved in here/checkpoint-3500/pytorch_model.bin
Savi

TrainOutput(global_step=5390, training_loss=0.09267741209500796, metrics={'train_runtime': 679.8774, 'train_samples_per_second': 126.558, 'train_steps_per_second': 7.928, 'total_flos': 4424507044177920.0, 'train_loss': 0.09267741209500796, 'epoch': 14.0})

In [52]:
import numpy as np
from datasets import load_metric

metric = load_metric("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)
trainer.evaluate()


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text_left, tuples, text_right.
***** Running Evaluation *****
  Num examples = 386
  Batch size = 8


{'eval_f1': 0.7602339181286548,
 'eval_loss': 1.7333605289459229,
 'eval_runtime': 1.0574,
 'eval_samples_per_second': 365.049,
 'eval_steps_per_second': 46.34}

## dirty_walmart_amazon

In [38]:
table_A = pd.read_csv(dirty_walmart_amazon+r'/deep_matcher/tableA.csv')
table_B = pd.read_csv(dirty_walmart_amazon+r'/deep_matcher/tableB.csv')
AG_train = pd.read_csv(dirty_walmart_amazon+r'/deep_matcher/train.csv')
AG_test = pd.read_csv(dirty_walmart_amazon+r'/deep_matcher/test.csv')
AG_valid = pd.read_csv(dirty_walmart_amazon+r'/deep_matcher/valid.csv')
AG_train, AG_test, AG_valid = merger(table_A,table_B,AG_train,AG_test,AG_valid)
AG_train.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1715: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,label,text_left,text_right,tuples
0,0,[CLS] [COL] title [VAL] elite screens cinewh...,[SEP] [COL] title [VAL] cinegray ezframe ser...,[CLS] [COL] title [VAL] elite screens cinewh...
1,0,[CLS] [COL] title [VAL] elite screens cinewh...,[SEP] [COL] title [VAL] powergain prime visi...,[CLS] [COL] title [VAL] elite screens cinewh...
2,0,[CLS] [COL] title [VAL] elite screens cinegr...,[SEP] [COL] title [VAL] powergain prime visi...,[CLS] [COL] title [VAL] elite screens cinegr...
3,0,[CLS] [COL] title [VAL] elite screens cinewh...,[SEP] [COL] title [VAL] cinewhite prime visi...,[CLS] [COL] title [VAL] elite screens cinewh...
4,0,[CLS] [COL] title [VAL] elite screens cinegr...,[SEP] [COL] title [VAL] cinewhite prime visi...,[CLS] [COL] title [VAL] elite screens cinegr...


In [39]:
AG_train = AG_train.groupby('label')
AG_train = AG_train.apply(lambda x: x.sample(AG_train.size().min()).reset_index(drop=True))
AG_train = AG_train.sample(frac=1).reset_index(drop=True)
AG_test = AG_test.groupby('label')
AG_test = AG_test.apply(lambda x: x.sample(AG_test.size().min()).reset_index(drop=True))
AG_test = AG_test.sample(frac=1).reset_index(drop=True)
AG_valid = AG_valid.groupby('label')
AG_valid = AG_valid.apply(lambda x: x.sample(AG_valid.size().min()).reset_index(drop=True))
AG_valid = AG_valid.sample(frac=1).reset_index(drop=True)

In [40]:
import os 
try:
  os.mkdir('merged_data/dirty_walmart_amazon/')
except FileExistsError:
  pass

AG_train.to_csv(r'merged_data/dirty_walmart_amazon/train.csv')
AG_test.to_csv(r'merged_data/dirty_walmart_amazon/test.csv')
AG_valid.to_csv(r'merged_data/dirty_walmart_amazon/valid.csv')


In [41]:
from datasets import load_dataset
raw_datasets = load_dataset('csv',data_files={'train':['merged_data/dirty_walmart_amazon/train.csv','merged_data/dirty_walmart_amazon/valid.csv'],
                                              'test':'merged_data/dirty_walmart_amazon/test.csv',
                                              })

Using custom data configuration default-be31ed2beda5c422


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-be31ed2beda5c422/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


In [42]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns('Unnamed: 0')
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'text_left', 'text_right', 'tuples'],
        num_rows: 6146
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'text_left', 'text_right', 'tuples'],
        num_rows: 386
    })
})

In [43]:
torch.cuda.empty_cache()
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-cased", num_labels=2)
training_args = TrainingArguments("savemodel",num_train_epochs=14,per_device_train_batch_size=16)

loading configuration file https://huggingface.co/distilbert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ebe1ea24d11aa664488b8de5b21e33989008ca78f207d4e30ec6350b693f073f.302bfd1b5e031cc1b17796e0b6e5b242ba2045d31d00f97589e12b458ebff27a
Model config DistilBertConfig {
  "activation": "gelu",
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.8.2",
  "vocab_size": 28996
}

loading weights file https://huggingface.co/distilbert-base-cased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/9c9f39769dba4c5fe379b4bc82973eb01297bd607954621434eb9f1bc85a23a0.06b428c87335c1bb22eae46fdab31c

In [44]:
tokenizer.decode(tokenized_datasets['train']['input_ids'][1])
trainer = Trainer(
    model=model, args=training_args, train_dataset=tokenized_datasets['train'], eval_dataset=tokenized_datasets['test']
)
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text_left, tuples, text_right.
***** Running training *****
  Num examples = 6146
  Num Epochs = 14
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 5390


Step,Training Loss
500,0.369100
1000,0.141000
1500,0.079900
2000,0.048300
2500,0.025900
3000,0.032800
3500,0.013700
4000,0.008400
4500,0.006400
5000,0.004300


Saving model checkpoint to savemodel/checkpoint-500
Configuration saved in savemodel/checkpoint-500/config.json
Model weights saved in savemodel/checkpoint-500/pytorch_model.bin
Saving model checkpoint to savemodel/checkpoint-1000
Configuration saved in savemodel/checkpoint-1000/config.json
Model weights saved in savemodel/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to savemodel/checkpoint-1500
Configuration saved in savemodel/checkpoint-1500/config.json
Model weights saved in savemodel/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to savemodel/checkpoint-2000
Configuration saved in savemodel/checkpoint-2000/config.json
Model weights saved in savemodel/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to savemodel/checkpoint-2500
Configuration saved in savemodel/checkpoint-2500/config.json
Model weights saved in savemodel/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to savemodel/checkpoint-3000
Configuration saved in savemodel/checkpoint-3

TrainOutput(global_step=5390, training_loss=0.06803487749400519, metrics={'train_runtime': 679.0108, 'train_samples_per_second': 126.72, 'train_steps_per_second': 7.938, 'total_flos': 4347061298571264.0, 'train_loss': 0.06803487749400519, 'epoch': 14.0})

In [45]:
import numpy as np
from datasets import load_metric

metric = load_metric("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text_left, tuples, text_right.
***** Running Evaluation *****
  Num examples = 386
  Batch size = 8


{'eval_f1': 0.8324022346368716,
 'eval_loss': 1.4343533515930176,
 'eval_runtime': 1.06,
 'eval_samples_per_second': 364.167,
 'eval_steps_per_second': 46.228}